<a href="https://colab.research.google.com/github/ensw1031/class-codes/blob/master/webscrapy_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#imports
from bs4 import BeautifulSoup, element
import re

import requests

In [85]:
#variaveis importantes

base_url = 'https://www.raulfulgencio.com.br'

apts = []

In [3]:
# regex

check_geo_regex = re.compile('(-)*\d{2}.\d{7}') #regex pras localizações (lat e long)
check_cep_regex = re.compile('\d{5}-\d{3}') #regex pra encontrar CEP

In [83]:
def extract_cep(name):
  # name = apt['name']
  url = f"https://www.google.com/search?q={name}+Londrina"
  # print(f"GET {url}")
  page = requests.get(url)
  soup = BeautifulSoup(page.content, 'html.parser')
  
  spans = soup.find_all('span')
  address = ''
  for sp in spans:  
    if(check_cep_regex.search(sp.text) is not None):
      address = sp.text

  cep = ''
  if address:
    aux = address.split(',')
    if(len(aux) >= 2 ):
      cep = aux[-2].strip()
      return cep
  
  return None

In [82]:
def extract_lat_long(cep):

  url = 'https://www.mapacep.com.br/index.php'
  # cep = '86020-100'
  payload = {
      'keywords': cep, 
      'sid': 'Busca+por+CEP%2C+Cidade%2C+Endere%C3%A7o%2C+CNPJ+ou+C%C3%B3d.+IBGE', 
      'submit': 'Pesquisar'
      }

  response = requests.post(url, data=payload)

  soup = BeautifulSoup(response.content, 'html.parser')

  div = soup.find('div', class_='addthis_inline_share_toolbox')

  if div is None:
    return None

  p = div.parent.find('p')

  loc = []
  for b in p.children:
    if(type(b) == element.NavigableString and check_geo_regex.match(b.strip()) is not None):
      loc.append(float(b.strip()))

  return(loc)



In [84]:
# função para extrair as informações do apt

def extract_apt_info(apt):
  price = apt.find('div', class_='price').find('span').text.strip()
  category = apt.find('div', class_='info').find('h3').find('a').text.strip()
  neighborhood = apt.find('div', class_='info').find('h3').find('small').text.strip()
  # print(f'{title} ({neighborhood}) {price}')
  # area = apt.find('i', class_='icon-area').parent.text.split(' ')[0]
  # bedrooms = apt.find('i', class_='icon-bedrooms')  
  # bathrooms = apt.find('i', class_='icon-bathrooms').parent.text.strio()
  # print(f'{title} ({neighborhood}) {price} - {area} ({bedrooms}, bathrooms)')
  detail_url = apt.find('a').get('href')
  # print(base_url+detail_url)
  print(f"GET {base_url}/{detail_url}")
  page = requests.get(f"{base_url}/{detail_url}")
  soup = BeautifulSoup(page.content, 'html.parser')
  name = soup.title.text.split(',')[0]
  amenities = soup.find_all('ul', class_='property-amenities-list col-md-6')
  apt = {
      'price': price,
      'category': category,
      'neighborhood': neighborhood,
      'name': name,
      'photo': apt['photo']
  }

  info_dict = dict()
  for col in amenities:
    infos = col.find_all('li')
    for info in infos:
      tokens = info.text.split(':')      
      key = tokens[0].strip().lower()
      value = tokens[1].strip()
      info_dict[key] = value
      # print(f"{key}: {value}")

  if('suítes' not in info_dict):
    info_dict['suítes'] = 0
    
  if('iptu' not in info_dict):
    info_dict['iptu'] = 0

  apt['info'] = info_dict
  apt['cep'] = extract_cep(name)

  if apt['cep'] is not None:
    apt['geo_ref'] = extract_lat_long(apt['cep'])

  return apt

In [98]:
x = range(1, 10)
count = 1
for n in x: 
  url = f"{base_url}/alugar/Londrina/Apartamento/Padrao/?pag={n}"
  print(f"GET {url}")
  page = requests.get(url)
  soup = BeautifulSoup(page.content, 'html.parser')
  apes = soup.find_all('div', class_='item col-sm-6 col-md-4 col-lg-3')
  for ape in apes:
    print(f'{count}')
    price = ape.find('div', class_='price').find('span').text.strip()
    ape['photo'] = ape.find('div', class_='image').find('img')['src']
    if('Consulte' not in price):
      extract_apt = extract_apt_info(ape)
      # print(extract_apt)
      apts.append(extract_apt)
      count = count + 1


GET https://www.raulfulgencio.com.br/alugar/Londrina/Apartamento/Padrao/?pag=1
1
GET https://www.raulfulgencio.com.br/alugar/Londrina/Apartamento/Padrao/Jardim-Agari/6749
2
GET https://www.raulfulgencio.com.br/alugar/Londrina/Apartamento/Padrao/Gleba-Fazenda-Palhano/6737
3
GET https://www.raulfulgencio.com.br/alugar/Londrina/Apartamento/Padrao/Jardim-Agari/6735
4
GET https://www.raulfulgencio.com.br/alugar/Londrina/Apartamento/Padrao/Jardim-Londrilar/6702
5
GET https://www.raulfulgencio.com.br/alugar/Londrina/Apartamento/Padrao/Gleba-Palhano/6697
6
GET https://www.raulfulgencio.com.br/alugar/Londrina/Apartamento/Padrao/Vila-Larsen-1/6687
7
GET https://www.raulfulgencio.com.br/alugar/Londrina/Apartamento/Padrao/Terra-Bonita/6676
8
GET https://www.raulfulgencio.com.br/alugar/Londrina/Apartamento/Padrao/Jamaica/6657
9
GET https://www.raulfulgencio.com.br/alugar/Londrina/Apartamento/Padrao/Jardim-Higienopolis/6645
10
GET https://www.raulfulgencio.com.br/alugar/Londrina/Apartamento/Padrao/G

In [99]:
len(apts)

104

In [74]:
aux = soup.find('blah')

True

In [88]:
apts[1]

{'category': 'Apartamento Padrão',
 'cep': '86050-464',
 'geo_ref': [-23.3284617, -51.1794095],
 'info': {'bairro': 'Gleba Fazenda Palhano',
  'banheiros': '2',
  'cidade': 'Londrina',
  'condomínio': '600,00 / Mês - (Sujeito a alterações)',
  'dormitórios': '2 \n                                \t\t\t\t\t\t\t\t\t\t\t/ Sendo 1 Suite',
  'iptu': 0,
  'operação': 'Locação',
  'suítes': '1',
  'tipo do imóvel': 'Padrão',
  'vagas na garagem': '2',
  'área útil': '93.57 M²'},
 'name': "Ed Fontaine D'or",
 'neighborhood': 'Gleba Fazenda Palhano',
 'photo': 'https://www.raulfulgencio.com.br/foto_thumb/2020/6737/londrina-apartamento-padrao-gleba-fazenda-palhano-11-11-2020_12-51-04-16.jpg',
 'price': '2.500,00'}

In [13]:
import branca
import jinja2
import folium

In [53]:
map = folium.Map(location=[-23.3071548,-51.1819446], tiles='Stamen Toner',)
map.zoom_start = 13

# marker_cluster = folium.MarkerCluster().add_to(map)

In [100]:

for apt in apts:
  if 'geo_ref' in apt and apt['geo_ref'] is not None:    
    folium.Marker(
      apt['geo_ref'], 
      popup=f"<img src=\"{apt['photo']}\" style=\"height: 64px\">", 
      icon=folium.Icon(color='darkblue', icon_color='white', icon='building-o', angle=0, prefix='fa')).add_to(map)

In [101]:
map